I know these are "unpythonic" but I eschewed some of Python's more elegant constructs in order to increase efficiency (for whatever that's worth)

This is just for fun and learning, there are more efficient methods for sorting.

In [1]:
%load_ext Cython
import random
import numpy as np

In [2]:
def merge_sort(L):
    """Merge sort:
    Recursively break the list into two sublists, sort those sublists, and merge the sorted lists.
    """

    def merge_sorted(L1, L2):
        """Merge 2 sorted lists"""

        count1 = len(L1)
        if count1 == 0:
            return L2

        count2 = len(L2)
        if count2 == 0:
            return L1
        elif count1 == 1 and count2 == 1:
            if L1[0] >= L2[0]:
                return [L2[0], L1[0]]
            else:
                return [L1[0], L2[0]]
        else:
            if L1[0] == L2[0]:
                return [L1[0], L2[0]] + merge_sorted(L1[1:], L2[1:])
            elif L1[0] > L2[0]:
                return [L2[0]] + merge_sorted(L1, L2[1:])
            else:
                return [L1[0]] + merge_sorted(L1[1:], L2)

    count = len(L)

    if count == 1:
        return L
    elif count == 2:
        return merge_sorted([L[0]], [L[1]])
    else:
        mid = int(count / 2)
        L1, L2 = L[:mid], L[mid:]
        return merge_sorted(merge_sort(L1), merge_sort(L2))

In [3]:
def quicksort(L):
    """Quicksort: 
    Recursively:
    Pick a pivot index (below uses the first item in the list). 
    Move items less than the pivot to the left.
    Move items greater than the pivot to the right.
    Mofidied: Keep items equal to pivot in the center with the pivot."""
    count = len(L)

    if count == 0:
        return []
    elif count == 1:
        return L
    elif count == 2:
        if L[0] <= L[1]:
            return L
        else:
            return L[::-1]
    else:
        pivot = L[0]
        left, middle, right = [], [], []

        for x in L:
            if x < pivot:
                left.append(x)
            elif x == pivot:
                middle.append(x)
            else:
                right.append(x)

        return quicksort(left) + middle + quicksort(right)

In [4]:
def selection_sort(L):
    """Selection sort:
    Recursively move the minimum item to the beginning of the list.
    Modified: Move all equal to minimum at the same time."""
    count = len(L)

    if count == 0:
        return []
    elif count == 1:
        return L
    else:
        minL = min(L)
        left, right = [], []

        for x in L:
            if x == minL:
                left.append(x)
            else:
                right.append(x)

        return left + selection_sort(right)

In [5]:
def insertion_sort(L):
    """Insertion sort:
    Create a new list one element at a time, putting each element where it belongs.
    
    TODO: Rewrite using binary search (or better?) instead of linear"""

    newL = []

    for x in L:
        if newL == []:
            newL = [x]
            continue
        newlen = len(newL)
        for i, y in enumerate(newL):
            if y >= x:
                newL = newL[:i] + [x] + newL[i:]
                break
            if i + 1 == newlen:
                newL.append(x)
                break

    return newL

In [6]:
def bubble_sort(L):
    """Bubble sort:
    Continuously swap adjacent pairs if they're in reverse order.
    Return the list when you make a full pass without any swaps."""
    is_sorted = 0
    count = len(L)

    if count == 0:
        return []
    elif count == 1:
        return L

    while is_sorted == 0:
        is_sorted = 1

        for i, x in enumerate(L):
            if i + 1 == count:
                break

            y = L[i + 1]

            if x > y:
                L[i], L[i + 1] = y, x
                is_sorted = 0

    return L

In [8]:
%%cython

from cython cimport sizeof
from libc.stdlib cimport malloc, free

cpdef list bubble_sort_cy(list L):
    """Bubble sort (cythonized):
    Continuously swap adjacent pairs if they're in reverse order.
    Return the list when you make a full pass without any swaps.
    
    Internal conversion between Python list and C array."""
    cdef:  # static type declarations
        int *c_list  # a C array of ints
        int count, i
        bint is_sorted
        
    count = len(L)
    c_list = <int *> malloc(count * sizeof(int))
    is_sorted = False
    
    #  convert Python list to C array
    for i in range(count):
        c_list[i] = L[i]
        
    # bubble sort
    while is_sorted == False:
        is_sorted = True
        
        for i in range(count):
            if i + 1 == count:
                break
            if c_list[i] > c_list[i+1]:
                c_list[i], c_list[i+1] = c_list[i+1], c_list[i]
                is_sorted = False
            
    
    #  convert C array back to Python list
    for i in range(count):
        L[i] = c_list[i]
        
    free(c_list)
    return L

In [9]:
%%time

L = [random.randint(0, 100000) for _ in range(5000)]

sorts = [
#    quicksort,
#    merge_sort,
#    selection_sort,
    bubble_sort,
    bubble_sort_cy,
#    insertion_sort,
]

for sort_f in sorts:
    #assert sorted(L) == sort_f(L), f"{sort_f.__name__} broken"
    
    print(f"{sort_f.__name__}:\t", end='')
    %timeit -n10000 sort_f(L)
    

bubble_sort:	1.03 ms ± 188 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
bubble_sort_cy:	138 µs ± 4.54 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Wall time: 1min 21s


In [10]:
insertion_sort(L)

[]